<h1>Algoritmo Genético - Trabajo IA 2018</h1>
Integrantes

- Sebastian Gaete
- Elard Koch
- Israel Díaz
- Francisco González

**notas**<br>
representación de individuo:
diccionario de python, el cual posee las siguientes claves,valores:

- **doors**: lista de puertas asociadas
- **ticks**: cantidad de ticks promedio del individuo
- **gen**: generación del individuo

una población de  individuos corresponderá a una lista de diccionarios

In [1]:
netlogo = None

In [2]:
import platform
import pyNetLogo
import random
import numpy as np

netlogo_gui=False #ejecutar netlogo con interfaz grafica (True = si, False = no)
current_gen = 0 #contador de la generación actual
max_gen=50 #total de generaciones
n_doors = 5 #numero de puertas a ubicar
pop_size=20 #tamaño poblacion
n_iterations=5 #cantidad de iteraciones por individuo
n_tournament=3 #cantidad de individuos por torneo
p_mut=0.3 #probabilidad de mutacion

plan_file="office.plan"

population = []
good_doors=[] #lista de coordenadas que pueden albergar una puerta 

- Para iniciar netlogo: **launch_netlogo()**
- Para iniciar netlogo: **kill_netlogo()**

In [3]:

def launch_netlogo():
    global netlogo
    print("Trying to start netlogo, please wait...")
    try:
        sistema=platform.system()
        if(sistema=='Windows'):
            netlogo = pyNetLogo.NetLogoLink(gui=netlogo_gui)
        elif(sistema=='Linux'):
            netlogo = pyNetLogo.NetLogoLink(gui=netlogo_gui,netlogo_home='NetLogo 6.0.4/')
        else:
            netlogo = pyNetLogo.NetLogoLink(gui=netlogo_gui)
        print("Done!")
    except Exception as e:
        print("Ocurrio un problema al levantar netlogo!")
        print(e)

def kill_netlogo():
    global netlogo
    if netlogo!=None:
        try:
            netlogo.kill_workspace()
            netlogo = None
        except Exception as e:
            print("An exception has ocurred while attempíng to kill NetLogo!")
            print(e)
    else:
        print("Please start netlogo first before kill it")

In [4]:
#iniciar netlogo
launch_netlogo()

Trying to start netlogo, please wait...
Done!


In [5]:
netlogo.load_model(r'escape4_v6.nlogo')
netlogo.command('set plan-data "{0}"'.format(plan_file))
netlogo.command("load-plan-file")

In [6]:
#detener netlogo
#kill_netlogo()

<h1>Funciones para algoritmo genético</h1>

In [207]:

#simulate recibe la lista ordenada de coordenadas de puertas, realiza la simulación correspondiente y retorna los ticks que ha demorado
#La distribución de puertas debe estar previamente validada, en caso de que las puertas no sean accesibles, la simulación podría quedar atascada en un while infinito
def simulate(doors):
    str_command = "set door-data {0}".format(str(doors).replace(',','').replace("[","[ ").replace("]"," ]")).replace("(","[").replace(")","]")
    netlogo.command(str_command)
    netlogo.command("show-plan")
    netlogo.command("generate-population")
    while(netlogo.report("count turtles")>0):
        netlogo.command("go")
    return(netlogo.report("ticks"))

def simulate_ind(individual):
    doors = list(map(lambda i:[good_doors[i][0],good_doors[i][1]],individual['doors']))
    return simulate(doors)
    
def get_good_doors(): #Obtener las coordenadas x e y de todas las puertas(posibles) que nos sirven
    x=list(map(lambda i:int(i),netlogo.report("listax")))
    y=list(map(lambda i:int(i),netlogo.report("listay")))
    return list(map(lambda i:(x[i],y[i]),range(len(x))))

def gen_zero(): #crea la generacion 0
    tmp_population=[]
    for i in range(pop_size):
        individual = {
            'doors':[],
            'gen':0,
            'ticks':0
        }
        tmp_population.append(individual)
        for j in range(n_doors):
            random_door_index = random.randint(0,len(good_doors))
            individual['doors'].append(random_door_index)
    return tmp_population
        

def evaluate_population(pop):
    for individual in pop:
        simulations_ticks=[]
        for i in range(n_iterations):
            simulations_ticks.append(simulate_ind(individual))
        individual['ticks'] = np.mean(simulations_ticks)
        
def do_tournament(pop,n_players):
    result = []
    tmp_pop = pop.copy()
    if(n_players > len(pop)):
        n_players=len(pop)
    while len(tmp_pop) >= n_players:
#        print("#")
#        print("len: {0}".format(len(tmp_pop)))
#        for i in tmp_pop:
#            print(i)
         
        players_selection = []
        while(len(players_selection)<n_players):
            selection = np.random.choice(tmp_pop)
            if(selection not in players_selection):
                players_selection.append(selection)
        #players_selection = list(map( lambda i: np.random.choice(tmp_pop) , range(n_players) ))
        
        players_doors=list(map(lambda p:p['doors'],players_selection))
        
        winner = players_selection[np.argmin([ i['ticks'] for i in players_selection])]
        tmp_pop=list(filter(lambda p:p['doors'] not in players_doors,tmp_pop))
#        print("players_selection")
        
#        for i in players_selection:
#            print(i)
        
#        print("winner")
#        print(winner)
        result.append(winner)
        
    if(len(tmp_pop)>0):
#        print("len: {0}".format(len(tmp_pop)))
#        print("#last")
        
#        for i in tmp_pop:
#            print(i)
        
        winner = tmp_pop[np.argmin([ i['ticks'] for i in tmp_pop])]
        result.append(winner)
#        print("players_selection")
#        for i in tmp_pop:
#            print(i)
#        print("winner")
#        print(winner)
        tmp_pop=list(filter(lambda p:p['doors'] not in players_doors,tmp_pop))
    return result


def do_crossover(pop):
    new_pop=[]
    punto_corte = random.randint(1,n_doors - 1)
    for i in range(len(pop)//2):
        parent1=pop[i]['doors']
        parent2=pop[len(pop)//2 + i]['doors']
        new_individual = parent1[:punto_corte] + parent2[punto_corte:]
        new_pop.append(new_individual)
        """
        print("#")
        print("pto_corte:",end=" ")
        print(punto_corte)
        print("padre1:",end=" ")
        print(parent1)
        print("padre2:",end=" ")
        print(parent2)
        print("hijo:",end=" ")
        print(new_individual)
        """
        
        
        

In [9]:
good_doors = get_good_doors() #guardar las puertas posibles en good_doors
len(good_doors)

298

<h1>Inicializar la población</h1>

In [11]:
#inicialización
population = gen_zero()
evaluate_population(population)
population

[{'doors': [214, 45, 267, 296, 52], 'gen': 0, 'ticks': 115.0},
 {'doors': [86, 190, 167, 28, 240], 'gen': 0, 'ticks': 70.8},
 {'doors': [141, 232, 204, 79, 52], 'gen': 0, 'ticks': 97.0},
 {'doors': [138, 254, 117, 52, 90], 'gen': 0, 'ticks': 156.2},
 {'doors': [103, 189, 38, 99, 37], 'gen': 0, 'ticks': 134.0},
 {'doors': [231, 216, 270, 34, 29], 'gen': 0, 'ticks': 108.2},
 {'doors': [132, 67, 232, 121, 131], 'gen': 0, 'ticks': 101.6},
 {'doors': [43, 244, 82, 134, 134], 'gen': 0, 'ticks': 123.6},
 {'doors': [101, 208, 285, 222, 147], 'gen': 0, 'ticks': 105.4},
 {'doors': [73, 241, 122, 208, 24], 'gen': 0, 'ticks': 91.2},
 {'doors': [255, 159, 211, 283, 60], 'gen': 0, 'ticks': 141.6},
 {'doors': [129, 73, 107, 247, 231], 'gen': 0, 'ticks': 165.0},
 {'doors': [51, 207, 239, 151, 86], 'gen': 0, 'ticks': 103.4},
 {'doors': [150, 175, 9, 104, 101], 'gen': 0, 'ticks': 147.0},
 {'doors': [51, 193, 103, 211, 58], 'gen': 0, 'ticks': 103.2},
 {'doors': [242, 28, 206, 213, 73], 'gen': 0, 'ticks':

In [205]:
winners = do_tournament(population,3)
do_crossover(winners)

#
pto_corte: 3
padre1: [194, 201, 165, 251, 39]
padre2: [214, 45, 267, 296, 52]
hijo: [194, 201, 165, 296, 52]
#
pto_corte: 3
padre1: [51, 207, 239, 151, 86]
padre2: [86, 190, 167, 28, 240]
hijo: [51, 207, 239, 28, 240]
#
pto_corte: 3
padre1: [242, 28, 206, 213, 73]
padre2: [73, 241, 122, 208, 24]
hijo: [242, 28, 206, 208, 24]


In [191]:
aa=30
a=[1,2,3,4,5]
b=[6,7,8,9,0]
a[:aa]+b[aa:]

[1, 2, 3, 4, 5]